# Hugging Face 本地管道

可以通过 `HuggingFacePipeline` 类在本地运行 Hugging Face 模型。

[Hugging Face Model Hub](https://huggingface.co/models) 托管了超过 120k 个模型、20k 个数据集和 50k 个演示应用程序 (Spaces)，所有这些都是开源且公开可用的，在一个在线平台上，人们可以轻松地一起进行协作和构建机器学习。

这些可以通过本地管道包装器从 LangChain 调用，或者通过 HuggingFaceHub 类调用其托管的推理端点来调用。

要使用，您应该安装 ``transformers`` Python [软件包](https://pypi.org/project/transformers/)，以及 [pytorch](https://pytorch.org/get-started/locally/)。您也可以安装 `xformer` 以获得更节省内存的注意力实现。

In [ ]:
%pip install --upgrade --quiet transformers

### 模型加载

模型可以通过 `from_model_id` 方法指定模型参数来加载。

In [ ]:
from langchain_huggingface.llms import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 10},
)

他们也可以通过直接传入现有的 `transformers` pipeline 来加载

In [ ]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
hf = HuggingFacePipeline(pipeline=pipe)

### 创建链

模型加载到内存后，您可以将其与提示组合起来形成一个链。

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

要在没有提示的情况下获得响应，您可以将 `skip_prompt=True` 与 LLM 绑定。

In [ ]:
chain = prompt | hf.bind(skip_prompt=True)

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

流式响应。

In [ ]:
for chunk in chain.stream(question):
    print(chunk, end="", flush=True)

### GPU 推理

当在带有 GPU 的机器上运行时，您可以指定 `device=n` 参数将模型置于指定的设备上。
默认值为 `-1`，表示 CPU 推理。

如果您有多个 GPU，或者模型过大无法装入单个 GPU，您可以指定 `device_map="auto"`。这需要并使用 [Accelerate](https://huggingface.co/docs/accelerate/index) 库来自动确定如何加载模型权重。

\*注意\*：`device` 和 `device_map` 不能同时指定，否则可能导致意外行为。

In [ ]:
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    device=0,  # replace with device_map="auto" to use the accelerate library.
    pipeline_kwargs={"max_new_tokens": 10},
)

gpu_chain = prompt | gpu_llm

question = "What is electroencephalography?"

print(gpu_chain.invoke({"question": question}))

### Batch GPU 推理

如果在带有 GPU 的设备上运行，您也可以在 GPU 上以批处理模式运行推理。

In [ ]:
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id="bigscience/bloom-1b7",
    task="text-generation",
    device=0,  # -1 for CPU
    batch_size=2,  # adjust as needed based on GPU map and model size.
    model_kwargs={"temperature": 0, "max_length": 64},
)

gpu_chain = prompt | gpu_llm.bind(stop=["\n\n"])

questions = []
for i in range(4):
    questions.append({"question": f"What is the number {i} in french?"})

answers = gpu_chain.batch(questions)
for answer in answers:
    print(answer)

### 使用 OpenVINO 后端进行推理

要使用 OpenVINO 部署模型，您可以指定 `backend="openvino"` 参数来触发 OpenVINO 作为后端推理框架。

如果您的设备有 Intel GPU，您可以指定 `model_kwargs={"device": "GPU"}` 在 GPU 上运行推理。

In [ ]:
%pip install --upgrade-strategy eager "optimum[openvino,nncf]" --quiet

In [ ]:
ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

ov_llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    backend="openvino",
    model_kwargs={"device": "CPU", "ov_config": ov_config},
    pipeline_kwargs={"max_new_tokens": 10},
)

ov_chain = prompt | ov_llm

question = "What is electroencephalography?"

print(ov_chain.invoke({"question": question}))

### 使用本地 OpenVINO 模型进行推理

您可以 [将您的模型导出](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#export) 为 OpenVINO IR 格式，并从本地文件夹加载模型。

In [ ]:
!optimum-cli export openvino --model gpt2 ov_model_dir

建议应用 8 位或 4 位权重量化，以使用 `--weight-format` 降低推理延迟和模型占用的空间：

In [ ]:
!optimum-cli export openvino --model gpt2  --weight-format int8 ov_model_dir # for 8-bit quantization

!optimum-cli export openvino --model gpt2  --weight-format int4 ov_model_dir # for 4-bit quantization

In [ ]:
ov_llm = HuggingFacePipeline.from_model_id(
    model_id="ov_model_dir",
    task="text-generation",
    backend="openvino",
    model_kwargs={"device": "CPU", "ov_config": ov_config},
    pipeline_kwargs={"max_new_tokens": 10},
)

ov_chain = prompt | ov_llm

question = "What is electroencephalography?"

print(ov_chain.invoke({"question": question}))

您可以通过激活和 KV 缓存量化的动态量化获得额外的推理速度改进。这些选项可以通过 `ov_config` 启用，如下所示：

In [ ]:
ov_config = {
    "KV_CACHE_PRECISION": "u8",
    "DYNAMIC_QUANTIZATION_GROUP_SIZE": "32",
    "PERFORMANCE_HINT": "LATENCY",
    "NUM_STREAMS": "1",
    "CACHE_DIR": "",
}

有关更多信息，请参阅 [OpenVINO LLM 指南](https://docs.openvino.ai/2024/learn-openvino/llm_inference_guide.html) 和 [OpenVINO 本地管道笔记本](/docs/integrations/llms/openvino/)。